# Agenda
1. Introduction
2. Business Questions
3. Business Assumptions
4. Solution Planning
5. Importing and Cleaning Data
6. Exploratory Data Analysis
   1. Hyphotesis Testing
   2. Answering Business Questions
7. Solutions
8.  Conclusion
9.  Next Steps

# 1) Introduction

This is a insight project designed to help an fictional CEO of a house sales company, located at King Count (USA). My aim is to draw htoughtful conclusions based on the houses dataset and business questions that he provided, validating or not business hyphotesis and generating maps and tables that can be used to assist him to take decisions based on data.

# 2) Business Questions

**We have two main questions:**
1. Which are the real states that House Rocket should buy, and how much will it cost?
2. What is the best moment to sell the real states, and how much will it cost?

# 3) Business Assumptions

+ We assumed that the season and region have an impact on the houses prices.
+ We considered old houses as the ones built before 1955.

# 4) Solution Planning

#### 4.1. Which are the real states that House Rocket should buy, and how much will it cost?

**Steps**
1. Collect the data on Kaggle
2. Group by region (zipcode)
3. For each region we should find the median price
4. Houses whose price is less than the median price, and is in good condition ($\geq 2$) should be bought by the suggested price
5. The final table to delivered will have the following structure:

|ID    | Region |Price | Median Price | Condition | Status |
|------|--------|-----------------|------------------|----------|--------|
|10330 |302349  |U\$ 450.000        |U\$ 500.000        |3         |Buy  |

#### 4.2. What is the best moment to sell the real states, and how much will it cost?

**Steps**
1. Group by region (zipcode) and by season
2. For each region + season group, we calculate the mean price
3. Sale conditions:
    + If the house price is bigger than the median of region + season
        + The sale price will be the purchase price + 10 %
    
    + If the house price is lower than the median of region + season
        +  The sale price will be the purchase price + 30 %
    
4. The final table to delivered will have the following structure:

|ID | Region | Season | Median Price | Purchase Price | Sale Price |Lucro| Condition |
|----------| ----------|----------|----------|---------- |----------|----------| ---------- |
|10330 | 302349 | Verão | U\$ 800.000,00 | U\$ 450.000,00 | U\$ 450.000,00 + 30\%| ??|

# 5) Importing and Cleaning Data

**Importing Libraries**

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

**Importing data**

In [2]:
house_rocket = pd.read_csv( 'kc_house_data.csv' )
house_rocket.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


**Displaying summary statistics**

In [3]:
house_rocket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [4]:
house_rocket.describe().round()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0
mean,4.580302e+09,540088.0,3.0,2.0,2080.0,15107.0,1.0,0.0,0.0,3.0,8.0,1788.0,292.0,1971.0,84.0,98078.0,48.0,-122.0,1987.0,12768.0
std,2.876566e+09,367127.0,1.0,1.0,918.0,41421.0,1.0,0.0,1.0,1.0,1.0,828.0,443.0,29.0,402.0,54.0,0.0,0.0,685.0,27304.0
min,1.000102e+06,75000.0,0.0,0.0,290.0,520.0,1.0,0.0,0.0,1.0,1.0,290.0,0.0,1900.0,0.0,98001.0,47.0,-123.0,399.0,651.0
25%,2.123049e+09,321950.0,3.0,2.0,1427.0,5040.0,1.0,0.0,0.0,3.0,7.0,1190.0,0.0,1951.0,0.0,98033.0,47.0,-122.0,1490.0,5100.0
50%,3.904930e+09,450000.0,3.0,2.0,1910.0,7618.0,2.0,0.0,0.0,3.0,7.0,1560.0,0.0,1975.0,0.0,98065.0,48.0,-122.0,1840.0,7620.0
75%,7.308900e+09,645000.0,4.0,2.0,2550.0,10688.0,2.0,0.0,0.0,4.0,8.0,2210.0,560.0,1997.0,0.0,98118.0,48.0,-122.0,2360.0,10083.0
max,9.900000e+09,7700000.0,33.0,8.0,13540.0,1651359.0,4.0,1.0,4.0,5.0,13.0,9410.0,4820.0,2015.0,2015.0,98199.0,48.0,-121.0,6210.0,871200.0


In [5]:
house_rocket.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016762,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798
price,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.001286,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.005160,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,-0.132109,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.018525,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,-0.002721,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.011592,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,-0.023783,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406


### 5.1 Transforming Data

In [6]:
# Converting 'date' column from object to datetime
house_rocket["date"] = pd.to_datetime( house_rocket["date"] )

In [7]:
# Adding new columns
house_rocket["year"] = house_rocket["date"].dt.year
house_rocket["month"] = house_rocket["date"].dt.month

print( house_rocket["year"].head(3) )
print( house_rocket["month"].head(3) )

0    2014
1    2014
2    2015
Name: year, dtype: int64
0    10
1    12
2     2
Name: month, dtype: int64


In [8]:
# Creating season column
house_rocket["season"] = house_rocket["date"].apply( lambda x: 'Spring' if ((x >= pd.to_datetime('2014-03-20')) & (x < pd.to_datetime('2014-06-21'))) | ((x >= pd.to_datetime('2015-03-20')) & (x < pd.to_datetime('2015-06-20'))) else
                                          'Summer' if (x >= pd.to_datetime('2014-06-21')) & (x < pd.to_datetime('2014-09-22'))  else
                                          'Fall' if (x >= pd.to_datetime('2014-09-22')) & (x < pd.to_datetime('2014-12-21')) else
                                          'Winter' )
house_rocket["season"].unique()

array(['Fall', 'Winter', 'Spring', 'Summer'], dtype=object)

### 5.2 Cleaning Data

**Checking for null values**

In [9]:
house_rocket.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
year             0
month            0
season           0
dtype: int64

**Checking for duplicated lines**

In [10]:
house_rocket.duplicated().sum()

0

# 6) Exploratory Data Analysis

## 6.1. Hyphotesis Testing

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

In [16]:
m_price_with_waterfront = house_rocket.loc[house_rocket["waterfront"] == 1].price.mean()
m_price_all = house_rocket.price.mean()

print( f'Média dos preço das casas com vista para água é de {m_price_with_waterfront:,.2f} dólares.' )
print( f'Média dos preço de todas as casas é de {m_price_all:,.2f} dólares.' )

Média dos preço das casas com vista para água é de 1,661,876.02 dólares.
Média dos preço de todas as casas é de 540,088.14 dólares.


In [22]:
porc_aumento_price = (m_price_with_waterfront / m_price_all * 100) - 100
print( f'Imóveis que possuem vista para água são {porc_aumento_price:.1f}% mais caros, na média.' )

Imóveis que possuem vista para água são 207.7% mais caros, na média.


#### **H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média..**

In [26]:
mean_price_1955 = house_rocket.loc[house_rocket["yr_built"] < 1955].price.mean()

print( f'Imóveis com data de construção menor que 1955 tem um preço médio de {mean_price_1955:,.2f} dólares.')

Imóveis com data de construção menor que 1955 tem um preço médio de 537,050.91 dólares.


In [27]:
porc_dec_1955 = 100 - ( mean_price_1955 / m_price_all * 100 )
print( f'Imóveis que possuem vista para água são {porc_dec_1955:.1f}% mais baratos, na média.' )

Imóveis que possuem vista para água são 0.6% mais baratos, na média.


#### **H3: Imóveis sem porão possuem área total(sqrt_lot), são 40% maiores do que os imóveis com porão.**

In [29]:
m_size_without_b = house_rocket.loc[house_rocket["sqft_basement"] == 0].sqft_lot.mean()
m_size_with_b = house_rocket.loc[house_rocket["sqft_basement"] > 0].sqft_lot.mean()
porc_increase_size = ( m_size_without_b / m_size_with_b * 100 ) - 100

print( f'Casas sem porão são {porc_increase_size:.2f}% maiores na média do que casas com porão.' )

Casas sem porão são 22.56% maiores na média do que casas com porão.


#### **H4: O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%.**

In [41]:
house_rocket.loc[:, ['price', 'year']].groupby( 'year' ).mean().reset_index()

,year,price
0,2014,539181.428415
1,2015,541988.992264


In [40]:
m_price_2014 = house_rocket.loc[house_rocket["year"] == 2014].price.mean()
m_price_2015 = house_rocket.loc[house_rocket["year"] == 2015].price.mean()
# porc_increse_yoy = 100 * (m_price_2015 - m_price_2014) / m_price_2015
porc_increse_yoy = 100 - (m_price_2014 / m_price_2015 * 100)

print( f'O cresimento YoY foi de apenas {porc_increse_yoy:.2f}% entre os anos 2014 e 2015.' )

O cresimento YoY foi de apenas 0.52% entre os anos 2014 e 2015.


In [42]:
# Standard deviation
std_price_2014 = house_rocket[house_rocket['year']==2014]['price'].std()
std_price_2015 = house_rocket[house_rocket['year']==2015]['price'].std()

In [43]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='2014',
    x=['Year'], y=[mean_price_2014],
    error_y=dict(type='data', array=[std_price_2014])
))
fig.add_trace(go.Bar(
    name='2015',
    x=['Year'], y=[mean_price_2015],
    error_y=dict(type='data', array=[std_price_2015])
))
fig.update_layout(barmode='group')
fig.show()

#### **H5: Imóveis com 3 banheiros tem um crescimento de MoM ( Month over Month ) de 15%.**

In [66]:
df_h5 = house_rocket.loc[house_rocket["bathrooms"] == 3, ['date', 'month', 'price', 'bathrooms']]
df_h5_m_prices = df_h5.loc[df_h5["bathrooms"] == 3].groupby( 'month' ).price.agg(['mean']).T

df_h5_cols = {1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"}

df_h5_m_prices.rename( df_h5_cols, axis=1, inplace=True )
df_h5_m_prices.head()

month,January,February,March,April,May,June,July,August,September,October,November,December
mean,672923.170732,696714.422222,768495.096154,709751.988235,689252.1,775057.564103,741405.042857,737576.5625,645246.887324,645523.015152,679910.465116,728341.1875


In [98]:
import plotly.express as px
fig = px.line(df_h5_m_prices.T)
fig.show()

In [101]:
MoM = []

for i in range(1, 11):
    val1 = df_h5_m_prices.T.iloc[i-1]["mean"]
    val2 = df_h5_m_prices.T.iloc[i]["mean"]

    result = round(100*(val2 - val1)/val1,2)
    MoM.append(result)

print( f'A média de crescimento dos preços MoM foi de {np.mean(MoM):.3f}%' )

A média de crescimento dos preços MoM foi de 0.375%


#### **H6: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

## 6.2. Answering Business Questions

# 7) Solutions

# 8)  Conclusion


# 9)  Next Steps